# MEME ANALYSIS PROJECT


In [54]:
#NIBTAHIL NAFEES
#21I-0330
#BS-AI-K

In [1]:
import numpy as np
import pandas as pd
import pickle
import nltk
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from PIL import Image
from PIL import ImageFile
import IPython.display as ipd
import warnings 
warnings.filterwarnings("ignore")
import skimage
from sklearn.svm import SVC
import sklearn
from skimage import filters
from skimage import feature
from skimage.io import imread, imshow
from skimage.feature import hog
import re
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier 
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
#from pytesseract import pytesseract
#import pytesseract as tess
#import easyocr
import gc
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES=True

### labels dataset

In [2]:
df_text=pd.read_csv(r"C:\Users\user\Desktop\i210330_paiproject\labels.csv")
df_text

,Unnamed: 0,image_name,text_ocr,text_corrected,overall_sentiment
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,very_positive
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,The best of #10 YearChallenge! Completed in le...,very_positive
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,Sam Thorne @Strippin ( Follow Follow Saw every...,positive
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 Year Challenge - Sweet Dee Edition,positive
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,neutral
...,...,...,...,...,...
6987,6987,image_6988.jpg,Tuesday is Mardi Gras Wednesday is Valentine's...,Tuesday is Mardi Gras Wednesday is Valentine's...,neutral
6988,6988,image_6989.jpg,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,MUST WATCH MOVIES OF 2017 ITI Chennai memes MA...,neutral
6989,6989,image_6990.png,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,LESS MORE TALKING PLANNING SODA JUNK FOOD COMP...,positive
6990,6990,image_6991.jpg,When I VERY have time is a fantasy No one has ...,When I have time is a fantasy. no one has time...,very_positive


In [3]:
df_text.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6992 entries, 0 to 6991
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         6992 non-null   int64 
 1   image_name         6992 non-null   object
 2   text_ocr           6831 non-null   object
 3   text_corrected     6987 non-null   object
 4   overall_sentiment  6992 non-null   object
dtypes: int64(1), object(4)
memory usage: 273.2+ KB


In [4]:
#filling dataset
df_text["text_corrected"][4799]='im gonna build some fancy walls ven though i have millions of extra dollars in gonna make the mexicans pay for it'
df_text["text_corrected"][6781]='if donald and hillary are together on a boat in the middle of the ocean and it sinks who survives america'
df_text["text_corrected"][6784]='bruh whi this tub of margarine look like Donald Trump'
df_text["text_corrected"][6786]='2016 election trump vs hillary still a better love story than twilight'

In [5]:
df_text=df_text.sort_values("image_name", axis = 0, ascending = True)

In [6]:
df_text

,Unnamed: 0,image_name,text_ocr,text_corrected,overall_sentiment
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,very_positive
9,9,image_10.png,FACEBOOK '10 YEAR CHALLENGE': A PLOY OR A SIMP...,FACEBOOK '10 YEAR CHALLENGE': A PLOY OR A SIMP...,positive
99,99,image_100.jpg,Drink water you may not meme generator.com,Drink water you may not meme-generator.com,neutral
999,999,image_1000.png,RT @BehindScenesPic: Martin Scorsese and Leona...,RT @BehindScenesPic: Martin Scorsese and Leona...,neutral
1000,1000,image_1001.png,Russian Leonardo DiCaprio omg,Russian Leonardo DiCaprio omg,positive
...,...,...,...,...,...
994,994,image_995.jpeg,DON'T LET YOUR DREAMS BE DREAMS! NOTHING IS IM...,DON'T LET YOUR DREAMS BE DREAMS! NOTHING IS IM...,positive
995,995,image_996.jpg,A TOAST TO THE MEMORY OF MY WORKOUT... CUZ I'M...,A TOAST TO THE MEMORY OF MY WORKOUT... CUZ I'M...,negative
996,996,image_997.jpg,RARE PICTURE OF LEONARDO DICAPRIO HOLDINGOSCAR...,RARE PICTURE OF LEONARDO DICAPRIO HOLDINGOSCAR...,very_positive
997,997,image_998.jpg,RIP WORLD'S LONGEST MEME 415 STARECAT.COM,RIP WORLD'S LONGEST MEME STARECAT.COM,positive


In [7]:
df_text=df_text.reset_index()
df_text

,index,Unnamed: 0,image_name,text_ocr,text_corrected,overall_sentiment
0,0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,very_positive
1,9,9,image_10.png,FACEBOOK '10 YEAR CHALLENGE': A PLOY OR A SIMP...,FACEBOOK '10 YEAR CHALLENGE': A PLOY OR A SIMP...,positive
2,99,99,image_100.jpg,Drink water you may not meme generator.com,Drink water you may not meme-generator.com,neutral
3,999,999,image_1000.png,RT @BehindScenesPic: Martin Scorsese and Leona...,RT @BehindScenesPic: Martin Scorsese and Leona...,neutral
4,1000,1000,image_1001.png,Russian Leonardo DiCaprio omg,Russian Leonardo DiCaprio omg,positive
...,...,...,...,...,...,...
6987,994,994,image_995.jpeg,DON'T LET YOUR DREAMS BE DREAMS! NOTHING IS IM...,DON'T LET YOUR DREAMS BE DREAMS! NOTHING IS IM...,positive
6988,995,995,image_996.jpg,A TOAST TO THE MEMORY OF MY WORKOUT... CUZ I'M...,A TOAST TO THE MEMORY OF MY WORKOUT... CUZ I'M...,negative
6989,996,996,image_997.jpg,RARE PICTURE OF LEONARDO DICAPRIO HOLDINGOSCAR...,RARE PICTURE OF LEONARDO DICAPRIO HOLDINGOSCAR...,very_positive
6990,997,997,image_998.jpg,RIP WORLD'S LONGEST MEME 415 STARECAT.COM,RIP WORLD'S LONGEST MEME STARECAT.COM,positive


In [8]:
#filling nan values with string 'emp'
df_text.fillna('emp',inplace=True)

In [9]:
for i in range(len(df_text)):
    if "emp" in df_text["text_corrected"][i]:
        df_text["text_corrected"][i] = df_text["text_ocr"][i]

In [10]:
len(df_text['text_corrected'])

6992

In [11]:
#Removing stopwords
import nltk
from nltk.corpus import stopwords
#nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
def remove_stopwords(word_list):
        processed_word_list = []        
        for i in word_list:
            if(i not in stop_words):
               processed_word_list .append(i)
        return processed_word_list
removed_stopwords = remove_stopwords(df_text['text_corrected'].to_list())
removed_stopwords

['LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIKUT TREND PLAY THE 10 YEARS CHALLENGE AT FACEBOOK imgflip.com',
 "FACEBOOK '10 YEAR CHALLENGE': A PLOY OR A SIMPLE MEME? mo by eXo Hạ Minh Le 18 Jan 2018",
 'Drink water you may not meme-generator.com',
 'RT @BehindScenesPic: Martin Scorsese and Leonardo DiCaprio on the set of The Departed https://t.co/RnXVYyHt44',
 'Russian Leonardo DiCaprio omg',
 "SOMETIMES I CRUNCH UP MY JIZZ WHEN IT DRIES THEN I BRING THE WHITE POWDER TO PARTIES AND WATCH PEOPLE SNORT MY COCK SAND What the fuck is this shit.... Meme DiCaprio all you want  but don't bring the Dude in to it.",
 "so y'all was really crying over leonardo dicaprios oscar but this man been in 200 movies over 50 years and y'all was silent 1 x 3 HOURS AGO Yoooo Word! This Man is a legend",
 'STEVE HARVEY SHOULD HOST THE OSCARS MAYBE LEONARDO DICAPRIO WILL WIN ONE OMG YES.',
 'THANKS EVERYONE VIA 9GAG.COM I HAVE BEEN PRACTICING MY SPEECH FOR 12 YEARS ALREADY MEMEFUL.COM',
 'THAT MOMENT YOU REA

In [12]:
len(removed_stopwords)

6992

In [13]:
df_text['text_corrected']==removed_stopwords
df_text['text_corrected']

0       LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...
1       FACEBOOK '10 YEAR CHALLENGE': A PLOY OR A SIMP...
2              Drink water you may not meme-generator.com
3       RT @BehindScenesPic: Martin Scorsese and Leona...
4                           Russian Leonardo DiCaprio omg
                              ...                        
6987    DON'T LET YOUR DREAMS BE DREAMS! NOTHING IS IM...
6988    A TOAST TO THE MEMORY OF MY WORKOUT... CUZ I'M...
6989    RARE PICTURE OF LEONARDO DICAPRIO HOLDINGOSCAR...
6990                RIP WORLD'S LONGEST MEME STARECAT.COM
6991    R.I.P. to the greatest meme of all time. The f...
Name: text_corrected, Length: 6992, dtype: object

In [14]:
#merging classes 
for x in range(len(df_text)):
    if df_text['overall_sentiment'][x]=='very_positive':
        df_text['overall_sentiment'][x]='positive'
    elif df_text['overall_sentiment'][x]=='very_negative':
        df_text['overall_sentiment'][x]='negative'

In [15]:
df_text['overall_sentiment']

0       positive
1       positive
2        neutral
3        neutral
4       positive
          ...   
6987    positive
6988    negative
6989    positive
6990    positive
6991     neutral
Name: overall_sentiment, Length: 6992, dtype: object

In [16]:
df_text['overall_sentiment'].unique()

array(['positive', 'neutral', 'negative'], dtype=object)

In [17]:
#assigning variables
order  ={"positive" : 1, "neutral" : 0, "negative" : -1}

In [18]:
df_text["overall_sentiment"] = df_text["overall_sentiment"].map(order)

In [19]:
#Normalizing variables
#le = LabelEncoder().fit_transform(df_text['overall_sentiment'])

In [20]:
tf = TfidfVectorizer(max_df=0.5) 
tfidf = tf.fit_transform(df_text["text_corrected"])

In [26]:
import sklearn
from sklearn.model_selection import train_test_split

In [27]:
tfidf

<6992x12931 sparse matrix of type '<class 'numpy.float64'>'
	with 91167 stored elements in Compressed Sparse Row format>

In [28]:
#le

In [29]:
print(list(tfidf))

[<1x12931 sparse matrix of type '<class 'numpy.float64'>'
	with 18 stored elements in Compressed Sparse Row format>, <1x12931 sparse matrix of type '<class 'numpy.float64'>'
	with 17 stored elements in Compressed Sparse Row format>, <1x12931 sparse matrix of type '<class 'numpy.float64'>'
	with 8 stored elements in Compressed Sparse Row format>, <1x12931 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored elements in Compressed Sparse Row format>, <1x12931 sparse matrix of type '<class 'numpy.float64'>'
	with 4 stored elements in Compressed Sparse Row format>, <1x12931 sparse matrix of type '<class 'numpy.float64'>'
	with 35 stored elements in Compressed Sparse Row format>, <1x12931 sparse matrix of type '<class 'numpy.float64'>'
	with 26 stored elements in Compressed Sparse Row format>, <1x12931 sparse matrix of type '<class 'numpy.float64'>'
	with 14 stored elements in Compressed Sparse Row format>, <1x12931 sparse matrix of type '<class 'numpy.float64'>'
	with 15 stored 

In [30]:
X_train, X_test, y_train, y_test = train_test_split(tfidf,df_text["overall_sentiment"], test_size=0.3, random_state=104,shuffle=True)

### Random Forest Classifier

In [31]:
text_classifier = RandomForestClassifier()
text_classifier.fit(X_train,y_train)
Predict = text_classifier.predict(X_test)
print("Confusion matrix", classification_report(y_test,Predict ))
print("Accuracy:", metrics.accuracy_score (y_test, Predict)*100)
print("F1-Score:", metrics.f1_score(y_test, Predict,average='weighted').mean()*100)
print("F1-Score macro:", metrics.f1_score(y_test, Predict,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

          -1       0.00      0.00      0.00       184
           0       0.42      0.05      0.10       698
           1       0.58      0.96      0.72      1216

    accuracy                           0.57      2098
   macro avg       0.33      0.34      0.27      2098
weighted avg       0.48      0.57      0.45      2098

Accuracy: 57.34032411820782
F1-Score: 45.148732987241516
F1-Score macro: 27.334972832655463


### Decision Tree Classifier

In [32]:
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)
pred_y=clf.predict(X_test)
print("Confusion matrix", classification_report(y_test,pred_y))
print('Accuracy:', accuracy_score(y_test,pred_y)*100)
print("F1-Score:", metrics.f1_score(y_test, pred_y,average=None).mean()*100)
print("F1-Score macro:", metrics.f1_score(y_test,pred_y,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

          -1       0.13      0.08      0.10       184
           0       0.34      0.28      0.31       698
           1       0.58      0.68      0.63      1216

    accuracy                           0.50      2098
   macro avg       0.35      0.35      0.34      2098
weighted avg       0.46      0.50      0.48      2098

Accuracy: 49.52335557673975
F1-Score: 34.473371973649506
F1-Score macro: 34.473371973649506


### Logistic Regression 

In [33]:
lr = LogisticRegression(solver='sag').fit(X_train,y_train)
pred2=lr.predict(X_test)
print("Confusion matrix", classification_report(y_test,pred_y))
print('Accuracy:', accuracy_score(y_test,pred2)*100)
print("F1-Score:", metrics.f1_score(y_test, pred2,average=None).mean()*100) 
print("F1-Score macro:", metrics.f1_score(y_test, pred2,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

          -1       0.13      0.08      0.10       184
           0       0.34      0.28      0.31       698
           1       0.58      0.68      0.63      1216

    accuracy                           0.50      2098
   macro avg       0.35      0.35      0.34      2098
weighted avg       0.46      0.50      0.48      2098

Accuracy: 57.00667302192565
F1-Score: 28.593288084886538
F1-Score macro: 28.593288084886538


## IMAGES DATASET

In [34]:
import skimage

In [35]:
path=(r"C:\Users\user\Desktop\i210330_paiproject\images\\")
li=os.listdir(path)
images=[]
i=0
for file in li:
    #z=imread(path+file,as_gray=True)
    z=skimage.transform.resize(imread(path+file,as_gray=True),(100,100))
    images.append(z)
    i=i+1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


1865
1866
1867
1868
1869
1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064


3508
3509
3510
3511
3512
3513
3514
3515
3516
3517
3518
3519
3520
3521
3522
3523
3524
3525
3526
3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707


5147
5148
5149
5150
5151
5152
5153
5154
5155
5156
5157
5158
5159
5160
5161
5162
5163
5164
5165
5166
5167
5168
5169
5170
5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346


6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988


In [36]:
Dataframe={'Name':li,'Images':images}

In [37]:
df=pd.DataFrame(Dataframe)
df

,Name,Images
0,image_1.jpg,"[[0.4128766205400558, 0.41224572579398944, 0.4..."
1,image_10.png,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,..."
2,image_100.jpg,"[[0.116763438293343, 0.11306334238026056, 0.10..."
3,image_1000.png,"[[0.2846914682299253, 0.2868378261296744, 0.28..."
4,image_1001.png,"[[0.9715638862606675, 0.9714675096530226, 0.97..."
...,...,...
6987,image_995.jpeg,"[[0.5911205309299019, 0.5926397697532422, 0.59..."
6988,image_996.jpg,"[[0.3494892657358355, 0.42102657341101696, 0.4..."
6989,image_997.jpg,"[[0.8424193718350534, 0.8754715828880489, 0.64..."
6990,image_998.jpg,"[[0.9999996032786497, 0.9999955080211308, 0.99..."


In [38]:
df['Flatten']=0
df['Flatten'][0]=np.array([1,2], dtype=object)

In [39]:
i=0
for i in range(len(df)):
    x=np.array(df['Images'][i])
    df['Flatten'][i]=x.flatten()
    i=i+1
    print(i)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


2949
2950
2951
2952
2953
2954
2955
2956
2957
2958
2959
2960
2961
2962
2963
2964
2965
2966
2967
2968
2969
2970
2971
2972
2973
2974
2975
2976
2977
2978
2979
2980
2981
2982
2983
2984
2985
2986
2987
2988
2989
2990
2991
2992
2993
2994
2995
2996
2997
2998
2999
3000
3001
3002
3003
3004
3005
3006
3007
3008
3009
3010
3011
3012
3013
3014
3015
3016
3017
3018
3019
3020
3021
3022
3023
3024
3025
3026
3027
3028
3029
3030
3031
3032
3033
3034
3035
3036
3037
3038
3039
3040
3041
3042
3043
3044
3045
3046
3047
3048
3049
3050
3051
3052
3053
3054
3055
3056
3057
3058
3059
3060
3061
3062
3063
3064
3065
3066
3067
3068
3069
3070
3071
3072
3073
3074
3075
3076
3077
3078
3079
3080
3081
3082
3083
3084
3085
3086
3087
3088
3089
3090
3091
3092
3093
3094
3095
3096
3097
3098
3099
3100
3101
3102
3103
3104
3105
3106
3107
3108
3109
3110
3111
3112
3113
3114
3115
3116
3117
3118
3119
3120
3121
3122
3123
3124
3125
3126
3127
3128
3129
3130
3131
3132
3133
3134
3135
3136
3137
3138
3139
3140
3141
3142
3143
3144
3145
3146
3147
3148


5625
5626
5627
5628
5629
5630
5631
5632
5633
5634
5635
5636
5637
5638
5639
5640
5641
5642
5643
5644
5645
5646
5647
5648
5649
5650
5651
5652
5653
5654
5655
5656
5657
5658
5659
5660
5661
5662
5663
5664
5665
5666
5667
5668
5669
5670
5671
5672
5673
5674
5675
5676
5677
5678
5679
5680
5681
5682
5683
5684
5685
5686
5687
5688
5689
5690
5691
5692
5693
5694
5695
5696
5697
5698
5699
5700
5701
5702
5703
5704
5705
5706
5707
5708
5709
5710
5711
5712
5713
5714
5715
5716
5717
5718
5719
5720
5721
5722
5723
5724
5725
5726
5727
5728
5729
5730
5731
5732
5733
5734
5735
5736
5737
5738
5739
5740
5741
5742
5743
5744
5745
5746
5747
5748
5749
5750
5751
5752
5753
5754
5755
5756
5757
5758
5759
5760
5761
5762
5763
5764
5765
5766
5767
5768
5769
5770
5771
5772
5773
5774
5775
5776
5777
5778
5779
5780
5781
5782
5783
5784
5785
5786
5787
5788
5789
5790
5791
5792
5793
5794
5795
5796
5797
5798
5799
5800
5801
5802
5803
5804
5805
5806
5807
5808
5809
5810
5811
5812
5813
5814
5815
5816
5817
5818
5819
5820
5821
5822
5823
5824


In [41]:
df

,Name,Images,Flatten
0,image_1.jpg,"[[0.4128766205400558, 0.41224572579398944, 0.4...","[0.4128766205400558, 0.41224572579398944, 0.41..."
1,image_10.png,"[[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
2,image_100.jpg,"[[0.116763438293343, 0.11306334238026056, 0.10...","[0.116763438293343, 0.11306334238026056, 0.108..."
3,image_1000.png,"[[0.2846914682299253, 0.2868378261296744, 0.28...","[0.2846914682299253, 0.2868378261296744, 0.283..."
4,image_1001.png,"[[0.9715638862606675, 0.9714675096530226, 0.97...","[0.9715638862606675, 0.9714675096530226, 0.971..."
...,...,...,...
6987,image_995.jpeg,"[[0.5911205309299019, 0.5926397697532422, 0.59...","[0.5911205309299019, 0.5926397697532422, 0.598..."
6988,image_996.jpg,"[[0.3494892657358355, 0.42102657341101696, 0.4...","[0.3494892657358355, 0.42102657341101696, 0.45..."
6989,image_997.jpg,"[[0.8424193718350534, 0.8754715828880489, 0.64...","[0.8424193718350534, 0.8754715828880489, 0.645..."
6990,image_998.jpg,"[[0.9999996032786497, 0.9999955080211308, 0.99...","[0.9999996032786497, 0.9999955080211308, 0.999..."


In [42]:
import sklearn
from sklearn.model_selection import train_test_split

In [43]:
#sorting values
df2 = df.sort_values('Name', ascending=True)

In [44]:
df2['labels']=df_text['overall_sentiment']

In [45]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [46]:
y=le.fit_transform(df2['labels'])

In [47]:
X= df2['Flatten']

In [48]:
len(y)

6992

In [49]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=104,shuffle=True)

### K Neighbors Classifier 

In [50]:
from sklearn.neighbors import KNeighborsClassifier
KNC=KNeighborsClassifier()
KNC.fit(list(X_train), list(y_train))
pr=KNC.predict(list(X_test))
print("Confusion matrix", classification_report(list(y_test),pr ))
print("Accuracy:", metrics.accuracy_score (list(y_test), pr)*100)
print("F1-Score:", metrics.f1_score(list(y_test), pr,average=None).mean()*100)
print("F1-Score Macro:", metrics.f1_score(list(y_test), pr,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

           0       0.05      0.03      0.04       184
           1       0.34      0.22      0.27       698
           2       0.59      0.75      0.66      1216

    accuracy                           0.51      2098
   macro avg       0.33      0.33      0.32      2098
weighted avg       0.46      0.51      0.48      2098

Accuracy: 51.28693994280267
F1-Score: 32.295425462966776
F1-Score Macro: 32.295425462966776


### SVC Classifier

In [51]:
svclassifier = SVC(kernel='linear')
svclassifier.fit(list(X_train), list(y_train))
pred_y = svclassifier.predict(list(X_test))
print("Confusion matrix", classification_report(list(y_test),pred_y ))
print("Accuracy:", metrics.accuracy_score (list(y_test), pred_y)*100)
print("F1-Score:", metrics.f1_score(list(y_test), pred_y,average=None).mean()*100)
print("F1-Score macro:", metrics.f1_score(list(y_test), pred_y,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

           0       0.10      0.14      0.11       184
           1       0.35      0.33      0.34       698
           2       0.58      0.56      0.57      1216

    accuracy                           0.44      2098
   macro avg       0.34      0.34      0.34      2098
weighted avg       0.46      0.44      0.45      2098

Accuracy: 44.4232602478551
F1-Score: 33.949709634052574
F1-Score macro: 33.949709634052574


### Gaussian Naive Bayes

In [52]:
nb = GaussianNB()
nb.fit(list(X_train), list(y_train))
Pred_y= nb.predict(list(X_test))
print("Confusion matrix", classification_report(list(y_test),Pred_y ))
print("Accuracy:", metrics.accuracy_score (list(y_test), Pred_y)*100)
print("F1-Score:", metrics.f1_score(list(y_test), Pred_y,average=None).mean()*100)
print("F1-Score macro:", metrics.f1_score(list(y_test), Pred_y,average='macro')*100)

Confusion matrix               precision    recall  f1-score   support

           0       0.09      0.37      0.15       184
           1       0.32      0.23      0.27       698
           2       0.57      0.40      0.47      1216

    accuracy                           0.35      2098
   macro avg       0.33      0.34      0.30      2098
weighted avg       0.45      0.35      0.38      2098

Accuracy: 34.50905624404194
F1-Score: 29.811242321527253
F1-Score macro: 29.811242321527253


### Overall f1 score

In [53]:
dic={'Random Forest Classifier':Predict,'Decision Tree Classifier':pred_y,'Logistic Regression':pred2,'KNeighborsClassifier':pr,'Svclassifier':pred_y,'GaussianNB':Pred_y}
df2=pd.DataFrame(dic)
df2

,Random Forest Classifier,Decision Tree Classifier,Logistic Regression,KNeighborsClassifier,Svclassifier,GaussianNB
0,1,1,1,2,1,0
1,1,1,1,2,1,2
2,1,2,1,2,2,0
3,1,2,1,2,2,2
4,1,2,1,2,2,1
...,...,...,...,...,...,...
2093,1,1,1,2,1,0
2094,1,2,1,2,2,1
2095,1,1,1,2,1,0
2096,0,2,1,2,2,0


In [54]:
pred_new=df2.mode(axis=1)
pred_new=pred_new[0]

In [55]:
pred_new

0       1.0
1       1.0
2       2.0
3       2.0
4       1.0
       ... 
2093    1.0
2094    1.0
2095    1.0
2096    2.0
2097    2.0
Name: 0, Length: 2098, dtype: float64

In [56]:
print("Accuracy:", metrics.accuracy_score (list(y_test),pred_new )*100)
print("F1-Score:", metrics.f1_score(list(y_test), pred_new,average='weighted').mean()*100)
print("F1-Score macro:", metrics.f1_score(list(y_test), pred_new,average='macro')*100)

Accuracy: 38.13155386081983
F1-Score: 38.80362883436568
F1-Score macro: 31.059648252717476


In [71]:
#Saving models
pickle.dump(text_classifier, open("Random_forest.pkl","wb"))
pickle.dump(clf, open("Decision_tree.pkl","wb"))
pickle.dump(lr, open("Logistic_Regression.pkl","wb"))
pickle.dump(KNC, open("KNeighbors_Classifier.pkl","wb"))
pickle.dump(svclassifier, open("SVC_Classifier.pkl","wb"))
pickle.dump(nb, open("GaussianNB.pkl","wb"))

In [66]:
x1=pickle.loads(model1)